# OneTwo and Vertex AI Reasoning Engine Demo

**Step 1: Setting Up Your Environment**


In [ ]:
!pip install git+https://github.com/google-deepmind/onetwo


**Setup tools**

In [ ]:
import requests

def get_exchange_rate(
   currency_from: str = "USD",
   currency_to: str = "EUR",
   currency_date: str = "latest",
):
   """Retrieves the exchange rate between two currencies on a specified date.
   Uses the Frankfurter API (https://api.frankfurter.app/) to obtain exchange rate data.
   Args:
       currency_from: The base currency (3-letter currency code). Defaults to "USD" (US Dollar).
       currency_to: The target currency (3-letter currency code). Defaults to "EUR" (Euro).
       currency_date: The date for which to retrieve the exchange rate. Defaults to "latest" for the most recent exchange rate data. Can be specified in YYYY-MM-DD format for historical rates.
   Returns:
       dict: A dictionary containing the exchange rate information.
            Example: {"amount": 1.0, "base": "USD", "date": "2023-11-24", "rates": {"EUR": 0.95534}}
   """
   response = requests.get(
       f"https://api.frankfurter.app/{currency_date}",
       params={"from": currency_from, "to": currency_to},
   )
   return response.json()




**Use this python function as a tool we need the following declaration:**

In [ ]:
from onetwo.stdlib.tool_use import llm_tool_use
exchange_rate_tool = llm_tool_use.Tool(
   name='get_exchange_rate',
   function=get_exchange_rate,
   ##example=EXAMPLES,
)

## Finish Tool
finish_tool = llm_tool_use.Tool(
   name='Finish',
   function=lambda x: x,
   description='Function for returning the final answer.',
)

**Step 2: Creating the OneTwo Agent with templates**

In [ ]:
from typing import Callable, Sequence
from onetwo.agents.react import ReActStep, ReActState

REACT_FEWSHOTS_EXCHANGE = [
   ReActState(
       inputs="What's the exchange rate from US dollars to British currency today??",
       updates=[
           ReActStep(
               thought=(
                   'The user wants to know the exchange rate between USD and GBP today'
               ),
               action=llm_tool_use.FunctionCall(
                   function_name='get_exchange_rate',
                   args=('USD','GBP','latest'),
                   kwargs={},
               ),
               observation='{"amount": 1.0, "base": "USD", "date": "2023-11-24", "rates": {"GBP": 0.95534}}',
               fmt=llm_tool_use.ArgumentFormat.JSON,
           ),
           ReActStep(
               is_finished=True,
               thought='The API response contains the exchange rate for today. The user should be informed about the current exchange rate. Extract the rate for GBP from the rates dictionary',
               action=llm_tool_use.FunctionCall(
                   function_name='Finish', args=('The exchange rate from USD to GBP today (2024-07-01) is 1 USD = 0.95534 GBP',), kwargs={}
               ),
               observation='The exchange rate from USD to GBP today (2024-07-01) is 1 USD = 0.95534 GBP',
               fmt=llm_tool_use.ArgumentFormat.PYTHON,
           ),
       ],
   ),
]

In [ ]:
class OneTwoAgent:
   def __init__(
           self,
           model: str,
           tools: Sequence[Callable],
           project: str,
           location: str,
       ):
       self.model_name = model
       self.tools = tools
       self.project = project
       self.location = location
   def set_up(self):
       """All unpickle-able logic should go here.
       The .set_up() method should not be called for an object that is being
       prepared for deployment.
       """
       import vertexai
       import os
       from onetwo.backends import vertexai_api
       from onetwo import ot
       from onetwo.builtins import llm
       from onetwo.agents import react
       from onetwo.stdlib.tool_use import llm_tool_use
       from onetwo.stdlib.tool_use import python_tool_use
       backend = vertexai_api.VertexAIAPI(generate_model_name=self.model_name)
       backend.register()

       self.react_agent = react.ReActAgent(
            exemplars=REACT_FEWSHOTS_EXCHANGE,
            environment_config=python_tool_use.PythonToolUseEnvironmentConfig(
            tools=self.tools,
           ),
           ##max_steps=20,
           ##stop_prefix=''
       )

   def query(self, input: str):
      """Query the application.
      Args:
          input: The user prompt.
      Returns:
          The output of querying the application with the given input.
      """
      from onetwo import ot
      answer = ot.run(self.react_agent(inputs=input))
      return {"answer": answer}

**Create an instance of our OneTwoAgent**

In [ ]:
agent = OneTwoAgent(
   model='gemini-1.5-flash-001',  # Required.
   tools=[exchange_rate_tool, finish_tool],  # Optional.
   project='PROJECT_ID',
   location='us-central1',
)
agent.set_up()

**Ask question from Agent**

In [28]:
response = agent.query(
     input="What is the exchange rate from US dollars to Polish currency?"
)

**Step 3: Deploying with Reasoning Engine**

In [ ]:
import vertexai
from vertexai.preview import reasoning_engines
PROJECT_ID = "PROJECT_ID"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
STAGING_BUCKET = "gs://vertex_staging_demo"  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)
remote_agent = reasoning_engines.ReasoningEngine.create(
    OneTwoAgent(
      model='gemini-1.5-flash-001',  # Required.
      tools=[exchange_rate_tool, finish_tool],  # Optional.
      project='genai-app-builder',
      location='us-central1',
   ),
   requirements=[
       "google-cloud-aiplatform==1.55.0",
       "pydantic",
       "requests",
       "blinker==1.4",
       "sentencepiece",
       "git+https://github.com/google-deepmind/onetwo"
   ],
   display_name="OneTwoAgent",
   description="A conversational agent that can answer questions about the world.",
)

INFO:vertexai.reasoning_engines._reasoning_engines:Using bucket vertex_staging_demo
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://vertex_staging_demo/reasoning_engine/reasoning_engine.pkl
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://vertex_staging_demo/reasoning_engine/requirements.txt
INFO:vertexai.reasoning_engines._reasoning_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://vertex_staging_demo/reasoning_engine/dependencies.tar.gz
INFO:vertexai.reasoning_engines._reasoning_engines:Creating ReasoningEngine
INFO:vertexai.reasoning_engines._reasoning_engines:Create ReasoningEngine backing LRO: projects/811614912259/locations/us-central1/reasoningEngines/5961657598917738496/operations/4811341009155784704
INFO:vertexai.reasoning_engines._reasoning_engines:ReasoningEngine created. Resource name: projects/811614912259/locations/us-central1/reasoningEngines/5961657598917738496
INF

**Querying the deployed agent**

In [ ]:
remote_agent.query(
input="What's the exchange rate from US dollars to Polish currency today?"
)

{'answer': 'The exchange rate from USD to PLN today (2024-09-02) is 1 USD = 3.8649 PLN'}